## MicroPython ESP32 Experimentation

### Establishing connection to target board
First, make sure you've got the right serial port. On unix-based systems, you can run `ls /dev/tty.*` to see your available serial devices. Replace as necessary below.

This will allow Jupyter (your host computer) to run commands and send/receive information to/from your target board in real time using the MicroPython REPL.

In [222]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200



***Connection broken [Device not configured]
You may need to reconnect
Closing serial Serial<id=0x7fb16768bca0, open=True>(port='/dev/tty.usbserial-0001', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [10]:
%sendtofile lib/runnera.py --source lib/runner.py

Sent 246 lines (8210 bytes) to lib/runnera.py.


In [11]:
%sendtofile lib/synthetica.py --source lib/synthetic.py

Sent 47 lines (1573 bytes) to lib/synthetica.py.


In [12]:
%sendtofile lib/decodinga.py --source lib/decoding.py

Sent 238 lines (8870 bytes) to lib/decodinga.py.


## Using a Runner for experimentation and logging
The a `Runner` is encapsulates the core functions in this EEG system, including peripheral setup, sampling, signal processing, logging and memory management. The `OnlineRunner` offers mostly the same functionality as the standard `Runner` class, except it allows for logging and other communication with a remote server - either on the Internet or on your local network.

### Offline functionality
The standard `Runner` is good for testing core functionality without the need for remote logging. See below for initialisation and execution.

In [6]:
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup() # setup peripherals and memory buffers

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498


### Calibration
If you are using an algorithm that leverages calibration data (MsetCCA, GCCA), you will need to record some calibration data to fit the decoder model. This is usually only done once off before inference starts. You may want to recalibrate at some semi-regular interval too though. 

At the moment, there is not an integrated process to record calibration data in the `Runner` class. You have to record calibration data and provide it to the runner which it will in turn use to fit its internal decoder model. In future, this will hopefully become more integrated and easy. For now, some random calibration data is generated below to illustrate the format which the runner/decoder expects. You need to provide iid calibration data trials for each stimulus frequency.

Note that if you try to run calibration using an incompatible algorithm (such as standard CCA), a warning will be generated and the calibration sequence will be skipped.

In [4]:
gc.collect()

In [7]:
from lib.synthetic import synth_X

calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
runner.calibrate(calibration_data)

In [82]:
print(gc.mem_free())

110224


### Decoding
When configured with a set of stimulus frequencies $\mathcal{F}=\{f_1, \dots, f_k, \dots, f_K\}$, the `Runner`'s decoder model consists of $K$ independent sub-classifiers $\Phi_k$ that each leverage the decoding algorithm selected. These independent classifiers must be calibrated independently. When the `Runner` is presented a new test observation, each sub-classifier $\Phi_k$ produces an output correlation estimate corresponding to $f_k$. Ultimately, the runner outputs a dictionary of frequency-correlation pairs of the form
```python
{f_1: 0.12, f_2: 0.03, f_3: 0.85}
```
The decoded output frequency is the one corresponding to the largest correlation in this output dictionary. In this example, it would be $f_3$.

In [9]:
test_freq = 7 # 7 Hz test signal
test_data = synth_X(test_freq, Nc, Ns, Nt=1)

print(runner.decoder.classify(test_data))

{12: 0.003198147, 10: 0.004834049, 7: 0.9978853}


In [223]:
from ulab import numpy as np
import utime as time
from lib.runner import Runner

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 256
Nt = 3
stim_freqs = [7, 10, 12]
decoding_period_s = 4

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

def preprocess_data(signal):
    
    """Preprocess incoming signal before decoding algorithms.
    This involves applying a bandpass filter to isolate the target SSVEP range
    and then downsampling the signal to the Nyquist boundary.
    
    Returns:
        [np.ndarray]: filtered and downsampled signal
    """
    from lib.signal import sos_filter
    downsample_freq = 64
    ds_factor = 256//downsample_freq
    return sos_filter(signal)[::ds_factor]

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
#     print(data)
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    
    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        trials.append(collectData(decode_period_s).flatten())
        print(trials)
    runner.stop()

    print(len(trials))

    gc.collect()
    return np.array(trials)


ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498


In [ ]:
#calibration_data = {f:getData(Nt, decoding_period_s) for f in stim_freqs}
calibration_data = {}
calibration_data[7] = getData(Nt, decoding_period_s)
gc.collect()

.

In [ ]:
calibration_data[10] = getData(Nt, decoding_period_s)
gc.collect()

In [ ]:
calibration_data[12] = getData(Nt, decoding_period_s)
gc.collect()

In [ ]:
print(gc.mem_free())

In [ ]:
decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)
print(gc.mem_free())

In [217]:
decode.run()

In [220]:
print(decode.decoder.classify(np.array(runner.output_buffer)))

{12: 0.6723066, 10: 0.2897166, 7: 0.02447039}


In [208]:
decode.stop()

In [221]:
print(calibration_data)

{12: array([[1.051727e-05, 0.3054571, 3.513627, ..., -12.86527, 5.246512, -4.119816],
       [-7.759659e-07, 0.1353391, 2.427785, ..., 15.21714, 14.3177, -12.74987],
       [7.617229e-06, 0.1164844, 0.1205384, ..., 10.95031, 10.45914, 2.580441]], dtype=float32), 10: array([[-1.159126e-06, 0.05807123, 2.746411, ..., -53.44389, 9.299017, 21.03572],
       [2.185002e-05, 0.07785761, -0.4632718, ..., 2.628769, 2.337142, 10.18596],
       [-3.123485e-06, -0.2111452, -4.57255, ..., 22.4221, 4.725843, 6.268902]], dtype=float32), 7: array([[-6.386206e-05, -0.6112373, -8.360441, ..., -4.869061, 6.936494, -8.043963],
       [-2.758059e-05, -0.2162395, -3.516753, ..., 24.15184, 26.8788, 10.28657],
       [2.853717e-06, 0.1219016, 1.436606, ..., -67.55063, -5.49646, 11.66945]], dtype=float32)}


### Asynchronous operation
Once the `Runner` has been configured and calibrated (if applicable), its internal `run()` loop can be started in which it will asynchronously sample and decode EEG data at preconfigured frequencies. Timing is handled using hardware timers on the ESP32 and interrupts are used to run asynchronous ISRs that handle sampling, preprocessing, filtering and decoding.

Note that once the async run loop has begun, you can still run commands or view the `Runner`'s attributes although there may be a noticeable delay since ISRs will typically get higher execution priority and there are quite a few interrupt loops running.

In [10]:
# start sampling and recording data (logging not setup in this case)
runner.run()

In [11]:
# see if runner has indeed started smapling
print(runner.is_sampling)

True


In [12]:
# display the contents of the output buffer - this will be updated internally by the runner
# at a rate determined by the sampling frequency and sample buffer size (typically every 1s)
print(runner.output_buffer)

[0.01507123, -0.1242969, -3.783963, -4.679721, 12.78181, 10.15671, -14.30577, 5.90723, 7.604455, -20.70899, 1.988277, 23.75715, 5.733761, -15.70858, -0.009125322, 7.742811, -16.0752, 1.230154, 25.41746, -0.5433288, -24.25103, 1.165697, 11.85342, -9.039201, 3.084062, 20.4376, -3.930146, -24.24293, 3.079892, 13.03915, -10.38186, 3.659443, 19.37393, -11.8056, -13.80852, 0.5107684, -7.631972, 19.07999, 11.34653, -23.30199, -5.879449, 11.77741, -0.7432878, -8.251669, 16.0036, 4.825533, -25.24554, -5.17767, 19.78535, 2.912423, -13.8064, 9.380515, 1.505076, -24.19353, -1.632591, 24.53315, 2.496613, -15.5124, 6.368776, 0.1071856, -18.20533, 1.534543, 20.90966, -1.599402, 0.01541734, 0.1648534, -0.7294103, -4.831893, -2.270404, 16.73792, 12.61396, -20.84267, -8.847502, 16.36489, -3.325031, -5.523794, 15.86657, 5.061106, -23.20341, -6.536437, 22.66838, 1.498738, -11.56312, 9.114537, 2.338871, -23.04785, -1.987546, 26.49001, 1.777297, -13.62673, 6.082272, 1.311298, -17.68964, 1.939265, 22.11768, 

In [15]:
# decode the contents of the output buffer. There will be a delay here if the runner 
# is currently running (i.e. `is_sampling=True`).
print(runner.decode())

{12: 0.00438, 10: 0.00906, 7: 0.03817}


In [16]:
# stop runner
runner.stop()

#### Simple decoding loop
In order to test online decoding, here is a basic synchronous loop-based option. Interrupt the cell to stop the infinite loop.

In [2]:
import utime as time
from lib.runner import Runner

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'

decode_period_s = 2 # read decoded output every x seconds

runner = Runner(decoding_algo, buffer_size=Ns) # initialise a base runner
runner.setup()

if decoding_algo in ['MsetCCA', 'GCCA']:
    from lib.synthetic import synth_X

    calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}
    runner.calibrate(calibration_data)

runner.run() # start async run loop

try:
    while True:
        time.sleep(decode_period_s)
        print(runner.decoded_output)
except KeyboardInterrupt:
    runner.stop()
    print('received SIGINT - stopping')

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62497708393011
{}
.{12: 0.15038, 10: 0.02075, 7: 0.11349}
{12: 0.15038, 10: 0.02075, 7: 0.11349}
.{12: 0.04253, 10: 0.02158, 7: 0.00613}
{12: 0.04253, 10: 0.02158, 7: 0.00613}
{12: 0.05438, 10: 0.00615, 7: 0.08}
.{12: 0.05438, 10: 0.00615, 7: 0.08}
{12: 0.0278, 10: 0.00456, 7: 0.02368}


*** Sending Ctrl-C

received SIGINT - stopping


### Testing your WiFi connection
In order to connect to a local WiFi network, you'll need to supply your network SSID and password in a `.env` file on the board. Doing this is easy: 
1. On your computer, create a `.env` file using `touch .env`. Update the `.env` file with the required fields:
    
    ```bash
    #.env 
    WIFI_SSID=<your network name>
    WIFI_PASSWORD=<your network password>
    
    ```
    
2. Send this file to your target device using the following command:
    ```ipython
%sendtofile --source lib/.env lib/.env  --binary
```

You may need to update the local (source) path to your `.env` file depending on where you created/stored it.

In [48]:
%sendtofile --source /Users/rishil/Desktop/.env lib/.env  --binary

Sent 55 bytes in 2 chunks to lib/.env.


In [67]:
from lib.utils import connect_wifi, load_env_vars
from lib.synthetic import synth_X

Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

# Here, we select the algorithm. Can be one of ['MsetCCA', 'GCCA', 'CCA']
decoding_algo = 'MsetCCA'
calibration_data = {f:synth_X(f, Nc, Ns, Nt=Nt) for f in stim_freqs}

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

network config: ('192.168.0.68', '255.255.255.0', '192.168.0.1', '194.168.4.100')


#### Online Runner
Now that you've established network connectivitiy, you can test out an `OnlineRunner`. In order to test web logging to a remote server, we can use a basic HTTP logger. However, this obviously needs an API/server willing to accept our requests. There is a basic logging API using `Flask` in `/eeg_lib/logging_server.py`. You can run it using `python logging_server.py` which will spin up a development server on the predefined port (5000 or 5001). Then, just configure your `OnlineRunner` with the appropriate logger params and you're set.

In [68]:
from lib.runner import OnlineRunner
from lib.logging import logger_types

api_host = "http://192.168.0.13:5001/" # make sure the port corresponds to your logging server configuration
log_params = dict(server=api_host, log_period=4, logger_type=logger_types.HTTP, send_raw=True, session_id='test_session_1')

runner = OnlineRunner(decoding_algo, buffer_size=Ns)
runner.setup(**log_params)

ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
network config: ('192.168.0.68', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [69]:
# start the runner - you should see requests being made to your local server
runner.calibrate(calibration_data)
runner.run()

In [70]:
runner.stop()

## Experimentation

In [155]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython d8a7bf8-dirty on 2022-02-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [12]:
%lsmagic

%capture [--quiet] [--QUIET] outputfilename
    records output to a file

%comment
    print this into output

%disconnect [--raw]
    disconnects from web/serial connection

%esptool [--port PORT] {erase,esp32,esp8266} [binfile]
    commands for flashing your esp-device

%fetchfile [--binary] [--print] [--load] [--quiet] [--QUIET]
                  sourcefilename [destinationfilename]
    fetch and save a file from the device

%ls [--recurse] [dirname]
    list files on the device

%lsmagic
    list magic commands

%mpy-cross [--set-exe SET_EXE] [pyfile]
    cross-compile a .py file to a .mpy file

%readbytes [--binary]
    does serial.read_all()

%rebootdevice
    reboots device

%sendtofile [--append] [--mkdir] [--binary] [--execute] [--source [SOURCE]] [--quiet]
                   [--QUIET]
                   [destinationfilename]
    send cell contents or file/direcectory to the device

%serialconnect [--raw] [--port PORT] [--baud BAUD] [--verbose]
    connects to a device over US